In [23]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
# import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [24]:
peak_hours_path = "2022-peak-hours-ca - 2022 Peak Hour Report.csv"
traffic_volumes_path = "2022 AADT DATA - 2022 AADT DATA.csv"

peak_hours_df = pd.read_csv(peak_hours_path)
aadt_df = pd.read_csv(traffic_volumes_path)

peak_hours_df.rename(columns={"RTE": "ROUTE", "CO": "COUNTY"}, inplace=True)

In [25]:
peak_hours_df.columns

Index(['DI', 'ROUTE', 'RTE_SFX', 'COUNTY', 'PM_PFX', 'PM', 'PM_SFX', 'CS',
       'LEG', 'YR', 'AM_DIR', 'AM_WAY_PHV', 'AM_K_FACTOR_AMT',
       'AM_D_FACTOR_AMT', 'AM_KD_FACTOR', 'AM_HOUR', 'AM_DAY', 'AM_MONTH',
       'PM_DIR', 'PM_WAY_PHV', 'PM_K_FACTOR_AMT', 'PM_D_FACTOR_AMT',
       'PM_KD_FACTOR', 'PM_HOUR', 'PM_DAY', 'PM_MONTH'],
      dtype='object')

In [26]:
peak_hours_df[["AM_DAY","PM_DAY","ROUTE","COUNTY","AM_HOUR","PM_HOUR","AM_MONTH","PM_MONTH"]]

,AM_DAY,PM_DAY,ROUTE,COUNTY,AM_HOUR,PM_HOUR,AM_MONTH,PM_MONTH
0,THU,THU,1,ORA,8,16,SEP,SEP
1,TUE,THU,1,SB,6,16,MAY,JUL
2,THU,THU,1,SB,6,17,NOV,NOV
3,SAT,FRI,1,SB,12,15,MAR,JUN
4,TUE,THU,1,SB,7,17,FEB,FEB
...,...,...,...,...,...,...,...,...
1335,TUE,THU,805,SD,8,16,MAY,APR
1336,FRI,THU,805,SD,11,15,JUN,MAY
1337,SAT,SAT,880,ALA,10,13,APR,MAY
1338,WED,FRI,905,SD,7,15,FEB,FEB


In [27]:
# morning_peak_hours_df = 
morning_peak_hours_df = peak_hours_df[["AM_DAY","AM_HOUR","AM_MONTH","AM_DIR","AM_WAY_PHV"]]
afternoon_peak_hours_df = peak_hours_df[["PM_DAY","PM_HOUR","PM_MONTH","PM_DIR","PM_WAY_PHV"]]

In [28]:
morning_peak_hours_df.head()

,AM_DAY,AM_HOUR,AM_MONTH,AM_DIR,AM_WAY_PHV
0,THU,8,SEP,S,2056
1,TUE,6,MAY,S,722
2,THU,6,NOV,S,924
3,SAT,12,MAR,S,523
4,TUE,7,FEB,N,1373


In [29]:
direction_map = {'N': 0, 'S': 1, 'E': 2, 'W': 3}

day_map = {
    'MON': 0, 'TUE': 1, 'WED': 2, 'THU': 3,
    'FRI': 4, 'SAT': 5, 'SUN': 6
}

month_map = {
    'JAN': 0, 'FEB': 1, 'MAR': 2, 'APR': 3,
    'MAY': 4, 'JUN': 5, 'JUL': 6, 'AUG': 7,
    'SEP': 8, 'OCT': 9, 'NOV': 10, 'DEC': 11
}

# encoding functions
def encode_direction(direction):
    return direction_map.get(direction.upper(), -1)

def encode_day(day):
    return day_map.get(day.upper(), -1)

def encode_month(month):
    return month_map.get(month.upper(), -1)

In [30]:
# aadt_df.head()
aadt_df.isnull().sum()
# aadt_df.shape

aadt_df.drop(columns=["ROUTE_SFX","PM_SFX","PM_PFX"], inplace=True)
aadt_df.dropna(subset=['AHEAD_PEAK_HOUR','BACK_PEAK_HOUR'],inplace=True)


In [34]:
aadt_df.head()

,DISTRICT,ROUTE,COUNTY,PM,LOCATION DESCRIPTION,BACK_PEAK_HOUR,BACK_PEAK_MADT,BACK_AADT,AHEAD_PEAK_HOUR,AHEAD_PEAK_MADT,AHEAD_AADT
1,12,1,ORA,0.780,"DANA POINT, DOHENY PARK ROAD",4250.0,42000.0,38900.0,4350.0,43500.0,38900.0
3,12,1,ORA,8.430,"LAGUNA BEACH, MOUNTAIN ROAD",3200.0,40500.0,37900.0,3200.0,40500.0,37900.0
4,12,1,ORA,9.418,"LAGUNA BEACH, JCT. RTE. 133 NORTH",3400.0,43000.0,38800.0,3850.0,43000.0,38800.0
5,12,1,ORA,9.600,"LAGUNA BEACH, CLIFF DRIVE/ ASTER STREET",3750.0,41000.0,38500.0,3750.0,41000.0,38500.0
6,12,1,ORA,11.500,"LAGUNA BEACH, NORTH CITY LIMITS",3550.0,39500.0,36800.0,3600.0,40500.0,36800.0


In [33]:
aadt_df.shape

(5924, 11)